## Code

In [1]:
%load_ext ipython_sparql_pandas

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
from random import randint
sparql = SPARQLWrapper('http://MacBook-Air-van-Daphne.local:7200/repositories/statements')
sparql.setReturnFormat(JSON)

In [3]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']

def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred = []
    listOfCompQ = []
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])
        listOfPred.append(attr['p'])
        listOfCompQ.append(attr['isCompound'])
    return(listOfObj,listOfPred,listOfCompQ)

def generateQuestion(listOfPred, listOfObj, listOfCompQ, index):
    flag = True
    
    if listOfCompQ[index]['value'] == 'false':  
        if ('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>') in AttrHistory:
            return generateQuestion(listOfPred, listOfObj, listOfCompQ, index+1)

        else:
            AttrHistory.append('<' + str(listOfPred[index]['value']) +'> <'+str(listOfObj[index]['value'])+ '>')
            
        userAnswer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} {listOfObj[index]["value"].split("/")[-1]}?' )
    
    if listOfCompQ[index]['value'] == 'true':  
        if ('<' + str(listOfPred[index]) +'> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <'+str(listOfObj[index])+ '>]') in AttrHistory:
            return generateQuestion(listOfPred, listOfObj, listOfCompQ, index+1)

        else:
            AttrHistory.append('<' + str(listOfPred[index]['value']) +'> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <'+ str(listOfObj[index]['value']) + '>]')
    
        userAnswer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} of type {listOfObj[index]["value"].split("/")[-1]}?' )
    
    
    try:
        int(listOfObj[index]['value'][0])
    except ValueError:
           flag = False
    
    
    if userAnswer.startswith('y'):   
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            
            if listOfCompQ[index]['value'] == 'false':
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) + '> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'.') 
            elif listOfCompQ[index]['value'] == 'true':
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) + '> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> "' + str(listOfObj[index]["value"]) + '"^^xsd:'+datatype+'].') 
        
        else:
            if listOfCompQ[index]['value'] == 'false':
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>.')
            elif listOfCompQ[index]['value'] == 'true':
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <'+str(listOfObj[index]["value"])+ '>].')
        
        
    elif userAnswer.startswith('n'):
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            
            if listOfCompQ[index]['value'] == 'false':
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
            elif listOfCompQ[index]['value'] == 'true':
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+']. }}')
        
        else:
            if listOfCompQ[index]['value'] == 'false':
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>. }}')   
            elif listOfCompQ[index]['value'] == 'true':
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <'+str(listOfObj[index]["value"])+ '>]. }}')   
     
    else:
        return generateQuestion(listOfPred, listOfObj, listOfCompQ, index+1)
                        
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)
  

In [4]:
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
        SELECT (count(distinct ?s) as ?count) ?p ?o ?isCompound WHERE {{
        {PosFilters}
        {{?s ?p [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?o] . BIND (true as ?isCompound) }} UNION {{?s ?p ?o . BIND (false as ?isCompound) }}
        {NegFilters}
        filter (?o != <http://www.w3.org/2002/07/owl#Class>)
        filter (?o != <http://example.com/popularEntity> )
        }}
        group by ?p ?o ?isCompound
        order by abs({left} - ?count )
        limit 5            
        """) 
    return FilterQuestion

In [5]:
def numberleft(PosFilters,NegFilters):
    query =  f"""
            select (count(distinct ?s) as ?count) where 
            {{
            {PosFilters}            
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))

In [6]:
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            
            {PosFilters}
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            
            }}
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres["results"]["bindings"]

In [7]:
# resets all variables
def game_reset():
    PosAttr= ['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
    NegAttr= ['']
    AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    left = numberleft("".join(PosAttr), "".join(NegAttr))
    FilterQuestion =  f"""
        SELECT (count(distinct ?s) as ?count) ?p ?o ?isCompound WHERE {{
        {{?s ?p [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?o] . BIND (true as ?isCompound) }} UNION {{?s ?p ?o . BIND (false as ?isCompound) }}
        ?s a <http://example.com/popularEntity> .
        filter (?o != <http://www.w3.org/2002/07/owl#Class>)
        filter (?o != <http://example.com/popularEntity> )
        }}
        group by ?p ?o ?isCompound
        order by abs( %s - ?count )
        limit 5
        """ %((left/2))
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred, listOfCompQ = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [9]:
#without printing results
import random

left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
i=0
outofguess = []

while i < 20:

    # list of Pred and Obj for questions
    listOfObj ,listOfPred, listOfCompQ = extractCountAndAttr(qres)

    # if only 1 popular entity is left break
    if left<=1:
        break

    # ask question and add filters to query
    print('Question %s' %(i+1) + ':')
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, listOfCompQ, 0)

    # number of popular entities left
    left = numberleft(PosFilters, NegFilters)

    # run query
    query = updateQuery(left/2,PosFilters, NegFilters)
    sparql.setQuery(query)
    qres = sparql.queryAndConvert()

    i+=1

# Prints answer
if i != 20:
    for attr in popentities(PosFilters, NegFilters):
        print(*[f'Your Answer is: {x.split("/")[-1]}' for x in [attr['s']['value']]][0:10]) 

        
# ran out of questions
else:
    print('Final Question:')
    for j in popentities(PosFilters, NegFilters):
            outofguess.append(f'{j[0].split("/")[-1][:40]:40s}')
    guess = random.choice(outofguess)
    print("Your answer is: " + guess)

Question 1:


Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type Person? y


Question 2:


Does the thing you are looking for have the attribute: knowsLanguage English_language? no


Question 3:


...

Question 12:


Does the thing you are looking for have the attribute: award of type Golden_Globe_Awards? no


Question 13:


Does the thing you are looking for have the attribute: award of type AACTA_Awards? no


Your Answer is: Basil_Rathbone
Your Answer is: Basil_Rathbone
Your Answer is: Basil_Rathbone
Your Answer is: Basil_Rathbone
